

#Load libraries

In [ ]:
#Set up google colab working environment
#!git clone https://github.com/zrosen3/Colombian-peso-counterfeit-detection.git
#%cd /content/Colombian-peso-counterfeit-detection/Code
#!pwd

In [4]:
import matplotlib.pyplot as plt #For plotting our visualizations
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import cv2
from PIL import Image
import seaborn as sns
from common_functions import LoadData, PeakData, MakeModel, ExtractLabels, ConfusionMatrix, EvaluateModel 


#Load in data

In [5]:
train_ds, test_ds, validation_ds, class_names =  LoadData()


Found 12479 files belonging to 13 classes.
Found 4160 files belonging to 13 classes.
Found 4160 files belonging to 13 classes.


## Run logistic regression model

In [6]:
#initialize model
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Dense(
        units = 13,
        activation = "softmax"
)])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=5
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


InvalidArgumentError: ignored

Evaluate model

In [7]:
EvaluateModel(model, test_ds, history)


NameError: ignored